In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

24/11/13 17:09:33 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.63 instead (on interface en0)
24/11/13 17:09:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/13 17:09:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.4.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


In [5]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [6]:
(film_category_df 
    .groupby('category_id')
    .agg({'film_id': 'count'})
    .join(category_df[['category_id', 'name']], how='left', on='category_id')
    .withColumnsRenamed({'name': 'category_name', 'count(film_id)': 'films_count'})
    [['category_name', 'films_count']]
    .sort(F.desc('films_count'))
    .show())

+-------------+-----------+
|category_name|films_count|
+-------------+-----------+
|       Sports|         74|
|      Foreign|         73|
|       Family|         69|
|  Documentary|         68|
|    Animation|         66|
|       Action|         64|
|          New|         63|
|        Drama|         62|
|        Games|         61|
|       Sci-Fi|         61|
|     Children|         60|
|       Comedy|         58|
|       Travel|         57|
|     Classics|         57|
|       Horror|         56|
|        Music|         51|
+-------------+-----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [7]:
(rental_df[['inventory_id']]
    .join(inventory_df[['inventory_id', 'film_id']], how='left', on='inventory_id')
    .join(film_actor_df[['actor_id', 'film_id']], how='left', on='film_id')
    .groupby('actor_id')
    .agg({'film_id': 'count'})
    .join(actor_df[['actor_id', 'first_name', 'last_name']], how='left', on='actor_id')
    .withColumnsRenamed({'count(film_id)': 'film_rentals'})
    [['first_name', 'last_name', 'film_rentals']]
    .sort(F.desc('film_rentals'))
    .limit(10)
    .show()
)

+----------+-----------+------------+
|first_name|  last_name|film_rentals|
+----------+-----------+------------+
|      GINA|  DEGENERES|         753|
|   MATTHEW|     CARREY|         678|
|      MARY|     KEITEL|         674|
|    ANGELA|WITHERSPOON|         654|
|    WALTER|       TORN|         640|
|     HENRY|      BERRY|         612|
|     JAYNE|      NOLTE|         611|
|       VAL|     BOLGER|         605|
|    SANDRA|     KILMER|         604|
|      SEAN|    GUINESS|         599|
+----------+-----------+------------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [8]:
(payment_df
     [['rental_id', 'amount']]
     .join(rental_df[['rental_id', 'inventory_id']], how='left', on='rental_id')
     .join(inventory_df[['inventory_id', 'film_id']], how='left', on='inventory_id')
     .join(film_category_df[['film_id', 'category_id']], how='left', on='film_id')
     .groupby('category_id')
     .agg({'amount': 'sum'})
     .join(category_df[['category_id', 'name']], how='left', on='category_id')
     .withColumnsRenamed({'sum(amount)': 'category_sales'})
     .sort(F.desc('category_sales'))
     .limit(1)
     [['name', 'category_sales']]
     .withColumn("category_sales", F.round('category_sales', 2))
     .show()
)

+------+--------------+
|  name|category_sales|
+------+--------------+
|Sports|       5314.21|
+------+--------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [9]:
(film_df
     [['film_id', 'title']]
     .join(inventory_df[['inventory_id', 'film_id']], how='left', on='film_id')
     .filter('inventory_id is Null')
     [['title']]
     .show()
)

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [10]:
(film_actor_df
     [['actor_id', 'film_id']]
     .join(film_category_df[['film_id', 'category_id']], on='film_id', how='left')
     .join(category_df[['category_id', 'name']], on='category_id', how='left')
     .filter('name == "Children"')
     .groupby('actor_id')
     .agg({'film_id': 'count'})
     .sort(F.desc('count(film_id)'))
     .limit(3)
     .join(actor_df[['actor_id', 'first_name', 'last_name']], on='actor_id', how='left')
     [['first_name', 'last_name']]
     .show()
)

+----------+---------+
|first_name|last_name|
+----------+---------+
|     HELEN|   VOIGHT|
|     RALPH|     CRUZ|
|    WHOOPI|     HURT|
+----------+---------+



Stop Spark session:

In [11]:
spark.stop()